In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
# Función para aplicar el filtro pasa alto
def high_pass_filter(image, kernel_size=3, sigma=0.25):
    # Aplica un filtro gaussiano (pasa bajo)
    gaussian_blur = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)
    
    # Elimina el componente de baja frecuencia para obtener el filtro pasa alto
    high_pass = image - gaussian_blur
    return high_pass

# Función para calcular la desviación estándar local
def local_standard_deviation(image, block_size=7):
    # Calcula la desviación estándar local de la imagen con una ventana deslizante
    mean = cv2.boxFilter(image, -1, (block_size, block_size))
    squared_mean = cv2.boxFilter(image**2, -1, (block_size, block_size))
    local_std = np.sqrt(squared_mean - mean**2)
    return local_std

# Función para calcular el mapa de nitidez
def sharpness_map(image, block_size=7, alpha=2.0):
    # Convierte la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplica el filtro pasa alto
    high_pass = high_pass_filter(gray_image)

    # Calcula la desviación estándar local
    local_std = local_standard_deviation(gray_image, block_size)

    # Pondera el filtro pasa alto por la desviación estándar
    high_freq_weighted = high_pass * local_std

    # Aplica un exponente
    sharpness_map = np.abs(np.log(np.abs(high_freq_weighted) + 1e-6) ** alpha)

    return sharpness_map

# Función para calcular el puntaje de nitidez de una imagen
def calculate_sharpness_score(image):
    sharp_map = sharpness_map(image)
    sharpness_score = np.max(sharp_map)  # Usamos el valor máximo del mapa de nitidez como el puntaje
    return sharpness_score

# Función para seleccionar la imagen más nítida de una carpeta de imágenes
def select_best_image_from_folder(folder_path):
    best_score = -1  # Inicializamos el mejor puntaje con un valor bajo
    best_image = None
    best_image_name = ""

    # Iteramos sobre todas las imágenes en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith((".jpg", ".png", ".jpeg")):  # Solo imágenes
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            
            # Calculamos el puntaje de nitidez
            score = calculate_sharpness_score(image)

            # Si encontramos una imagen con un puntaje mayor, la seleccionamos como la mejor
            if score > best_score:
                best_score = score
                best_image = image
                best_image_name = filename

    return best_image, best_image_name, best_score

# Función para mostrar la imagen y su mapa de nitidez
def display_sharpness(image, sharp_map):
    plt.figure(figsize=(10, 5))
    
    # Mostrar la imagen original
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Imagen Original")
    plt.axis('off')

    # Mostrar el mapa de nitidez
    plt.subplot(1, 2, 2)
    plt.imshow(sharp_map, cmap='hot')
    plt.title("Mapa de Nitidez")
    plt.axis('off')
    
    plt.show()

In [ ]:
# Ruta de la carpeta que contiene las imágenes (ajustar esta ruta)
folder_path = '/images/000002' # Cambia esto a la ruta de tu carpeta de imágenes

# Seleccionamos la mejor imagen
best_image, best_image_name, best_score = select_best_image_from_folder(folder_path)

# Mostrar la mejor imagen y su mapa de nitidez
if best_image is not None:
    print(f"La imagen más nítida es: {best_image_name} con un puntaje de nitidez de {best_score:.4f}")
    sharp_map = sharpness_map(best_image)
    display_sharpness(best_image, sharp_map)
else:
    print("No se encontraron imágenes en la carpeta.")
/